# **3390. Longest Team Pass Streak**

**Table: Teams**
``` sql
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| player_id   | int     |
| team_name   | varchar | 
+-------------+---------+
```
player_id is the unique key for this table.

Each row contains the unique identifier for player and the name of one of the teams participating in that match.

**Table: Passes**
``` sql
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| pass_from   | int     |
| time_stamp  | varchar |
| pass_to     | int     |
+-------------+---------+
```
(pass_from, time_stamp) is the unique key for this table.

pass_from is a foreign key to player_id from Teams table.

Each row represents a pass made during a match, time_stamp represents the time in minutes (00:00-90:00) when the pass was made,

pass_to is the player_id of the player receiving the pass.

Write a solution to find the longest successful pass streak for each team during the match. The rules are as follows:

A successful pass streak is defined as consecutive passes where:

Both the pass_from and pass_to players belong to the same team

A streak breaks when either:

The pass is intercepted (received by a player from the opposing team)

Return the result table ordered by team_name in ascending order.

The result format is in the following example.

 

**Example:**

**Input:**

**Teams table:**
``` sql
+-----------+-----------+
| player_id | team_name |
+-----------+-----------+
| 1         | Arsenal   |
| 2         | Arsenal   |
| 3         | Arsenal   |
| 4         | Arsenal   |
| 5         | Chelsea   |
| 6         | Chelsea   |
| 7         | Chelsea   |
| 8         | Chelsea   |
+-----------+-----------+
```
**Passes table:**
``` sql
+-----------+------------+---------+
| pass_from | time_stamp | pass_to |
+-----------+------------+---------+
| 1         | 00:05      | 2       |
| 2         | 00:07      | 3       |
| 3         | 00:08      | 4       |
| 4         | 00:10      | 5       |
| 6         | 00:15      | 7       |
| 7         | 00:17      | 8       |
| 8         | 00:20      | 6       |
| 6         | 00:22      | 5       |
| 1         | 00:25      | 2       |
| 2         | 00:27      | 3       |
+-----------+------------+---------+
```
**Output:**
``` sql
+-----------+----------------+
| team_name | longest_streak |
+-----------+----------------+
| Arsenal   | 3              |
| Chelsea   | 4              |
+-----------+----------------+
```
**Explanation:**

**Arsenal's streaks:**
- First streak: 3 passes (1→2→3→4) ended when player 4 passed to Chelsea's player 5

**Second streak: 2 passes (1→2→3)**
- Longest streak = 3

**Chelsea's streaks:**

First streak: 3 passes (6→7→8→6→5)

Longest streak = 4

**Solution-1:**
``` sql
WITH TeamPasses (team, complete, time_stamp) AS (
    SELECT A.team_name,
        CASE WHEN A.team_name = C.team_name THEN 1 ELSE 0 END,
        B.time_stamp
    FROM Teams AS A
    INNER JOIN Passes AS B
        ON B.pass_from = A.player_id
    INNER JOIN Teams AS C
        ON C.player_id = B.pass_to
),
Islands (team, complete, island) AS (
    SELECT team, 
        complete, 
        ROW_NUMBER() OVER(PARTITION BY team ORDER BY time_stamp)
            - ROW_NUMBER() OVER(PARTITION BY team, complete ORDER BY time_stamp)
    FROM TeamPasses
),
IslandSize (team, island, island_size) AS (
    SELECT team, island, COUNT(*)
    FROM Islands
    WHERE complete = 1
    GROUP BY team, island
)
SELECT team AS team_name,
    MAX(island_size) AS longest_streak
FROM IslandSize
GROUP BY team

**Solution-2:**
``` sql
WITH TeamPasses AS (
    SELECT A.team_name as team,
        CASE WHEN A.team_name = C.team_name THEN 1 ELSE 0 END as complete,
        B.time_stamp
    FROM Teams AS A
    INNER JOIN Passes AS B
        ON B.pass_from = A.player_id
    INNER JOIN Teams AS C
        ON C.player_id = B.pass_to
),
Islands AS (
    SELECT team, 
        complete, 
        ROW_NUMBER() OVER(PARTITION BY team ORDER BY time_stamp)
            - ROW_NUMBER() OVER(PARTITION BY team, complete ORDER BY time_stamp) as island
    FROM TeamPasses
),
IslandSize AS (
    SELECT team, island, COUNT(*) as island_size
    FROM Islands
    WHERE complete = 1
    GROUP BY team, island
)
SELECT team AS team_name,
    MAX(island_size) AS longest_streak
FROM IslandSize
GROUP BY team